In [ ]:
params = {"figure.figsize": (12,9),
          "font.size": 20,
          "font.weight": "normal",
          "xtick.major.size": 9,
          "xtick.minor.size": 4,
          "ytick.major.size": 9,
          "ytick.minor.size": 4,
          "xtick.major.width": 4,
          "xtick.minor.width": 3,
          "ytick.major.width": 4,
          "ytick.minor.width": 3,
          "xtick.major.pad": 8,
          "xtick.minor.pad": 8,
          "ytick.major.pad": 8,
          "ytick.minor.pad": 8,
          "lines.linewidth": 3,
          "lines.markersize": 10,
          "axes.linewidth": 4,
          "legend.loc": "best",
          "text.usetex": False,    
          "xtick.labelsize" : 20,
          "ytick.labelsize" : 20,
          }

import matplotlib
matplotlib.rcParams.update(params)

In [ ]:
import astroalign as aa

import numpy as np
import glob
from astropy import units as un
from astropy.coordinates import SkyCoord
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as spstats

from matplotlib.offsetbox import AnchoredText
import matplotlib.gridspec as gridspec

from astropy.io import fits

from astroquery.vizier import Vizier

%matplotlib inline

In [ ]:
def get_transformation(target_coords, field_coords, min_sep=5./60./60., print_outs=False):
    '''
    Find the transformation matrix to adjust your field coordinates to
    match the target coordinates.
    
    This is a slightly different version for the MC investigation
    of the uncertainties.
    
    Here we have some target coordinates, which are the coordinates
    of the sources that you know have good astrometry. Then you have
    the coordinates of every source in your field. These are the
    coordinates that you want to transform such that they match
    the target coordinates. This code finds the transformation
    matrix to shift all of your field coordinates to match
    the target coordinates.
    
    Args:
    target_coords (SkyCoord): a SkyCoord object with the coordinates
                              of the sources that have known
                              astrometric accuracy
    field_coords (SkyCoord): a SkyCoord object with the coordinates
                             of the sources that do not have
                             known astrometric accuracy
    kwargs:
    min_sep (float): the minimum separation (in degrees)
                     for a match between your target source
                     coordinates and a source in your
                     field coordinates
                     Default: 5./60./60. degrees
    print_outs (bool): if you want to print out some
                       information about the matches
                       as you go. True means the info
                       will be printed.
                       Default: False
    Returns:
    (transform, [np.nanmean(original_separations),
                np.nanmean(new_separations),
                np.nanmedian(original_separations),
                np.nanmedian(new_separations),
                np.nanmax(original_separations),
                np.nanmax(new_separations)],
    transform(field))

    transform: this is the function to transform your
               field_coords to match the target_coords
    original_separations is the separation between your
    target sources and field sources before transformation.
    new_separations is the separation between your
    target sources and field sources after transformation.
    transform (field): gives the transformed coordinates
                       of the reference MeerKAT sources
    '''    
    matches = []
    # find which sources in your field_sources
    # match the target_coords
    for t, tg in enumerate(target_coords):
        seps = tg.separation(field_coords)
        if np.nanmin(seps.deg) < min_sep:
            matches.append([tg.ra.deg, tg.dec.deg,
                            field_coords[np.nanargmin(seps.deg)].ra.deg,
                            field_coords[np.nanargmin(seps.deg)].dec.deg,
                            np.nanmin(seps.deg)*60.*60.])

    matches = np.array(matches)
    target = matches[:, :2]
    field = matches[:, 2:4]

    n = field.shape[0]
    # Small functions to pad arrays withs ones
    pad = lambda x: np.hstack([x, np.ones((x.shape[0], 1))])
    unpad = lambda x: x[:,:-1]

    # Pad your coordinates
    X = pad(field)
    Y = pad(target)

    # Solve the linear least squares
    # problem of transforming the X coords
    # into the Y coords
    A, res, rank, s = np.linalg.lstsq(X, Y)

    # Make the function to transform
    # your X coords to match the
    # Y coord frame
    transform = lambda x: unpad(np.dot(pad(x), A))

    # Work out whether the field sources have
    # actually moved closer to the target
    # sources after transformation
    field_sc = SkyCoord(field, unit=(un.deg, un.deg))
    field_sc_T = SkyCoord(transform(field), unit=(un.deg, un.deg))
    target_sc = SkyCoord(target, unit=(un.deg, un.deg))
    original_separations = []
    new_separations = []
    for m, mkt in enumerate(field_sc):
        sep0 = mkt.separation(target_sc[m])
        original_separations.append(sep0.deg*60.*60.)
        sep1 = field_sc_T[m].separation(target_sc[m])
        new_separations.append(sep1.deg*60.*60.)
    original_separations = np.array(original_separations)
    new_separations = np.array(new_separations)
    
    if print_outs:
        # Print some useful, quick look values]
        print(('Separation between the '
               'ATPMN reference souces '
               'and MeerKAT reference\n'
               'sources in arcsec before '
               '(left column) and after '
               '(right column) transformation.'))
        print(np.hstack((matches,
                         np.expand_dims(new_separations,
                                        axis=1)))[:, 4:])    
        print(('Original mean sep: {0:.4f}", '
               'new mean sep: {1:.4f}"').format(np.nanmean(original_separations),
                                                np.nanmean(new_separations)))
        print(('Original median sep: {0:.4f}", '
               'new median sep: {1:.4f}"').format(np.nanmedian(original_separations),
                                                  np.nanmedian(new_separations)))
    
    return (transform, [np.nanmean(original_separations),
                        np.nanmean(new_separations),
                        np.nanmedian(original_separations),
                        np.nanmedian(new_separations),
                        np.nanmax(original_separations),
                        np.nanmax(new_separations)],
            transform(field))

## Monte Carlo uncertainties

We want to determine the dominant source of uncertainty for the astrometry of the sources in the GX 339-4 field. We know that the median astrometric uncertainty for the ATPMN sources in 0.4 arcsec, and we have 6 ATPMN sources that match MeerKAT sources in the GX 339-4 field. So we are going to scatter the ATPMN sources usin an uncertainty of 0.4 arcsec and see how that affects the position and scatter of the MeerKAT sources after transforming them.

In [ ]:
# Find the ATPMN sources that are in the GX 339 FoV
# using Vizier
result_atpmn = Vizier.query_region(SkyCoord(ra=255.706, dec=-48.790,
                                            unit=(un.deg, un.deg),
                                            frame='icrs'),
                             width="180m",
                             catalog=["J/MNRAS/422/1527/atpmncat"])

# Remove the ATPMN sources that match extended sources
# in the GX 339-4 field, as these won't have accurate
# positions
remove_sources = SkyCoord(np.array([254.57271, 254.56767,
                                    254.41062, 257.15887])*un.deg,
                          np.array([-47.51383, -47.51764,
                                    -50.19133, -49.00014])*un.deg)
ras = []
decs = []
for ra in result_atpmn[0]['RAJ2000']:
    ras.append(ra)
for dec in result_atpmn[0]['DEJ2000']:
    decs.append(dec)

# Get the ATPMN coordinates of the sources
# that you're going to use for astrometry
og_atpmn_coords = SkyCoord(ras, decs, unit=(un.hourangle, un.deg))
atpmn_coords = []
for a, atpmn in enumerate(og_atpmn_coords):
    r_seps = atpmn.separation(remove_sources)
    if np.nanmin(r_seps.deg) > 10./60./60.:
        atpmn_coords.append([atpmn.ra.deg, atpmn.dec.deg])
# Add the GX 339 and pulsar coordinates
# to your astrometry coordinates
astro_coords = np.array(atpmn_coords)

# Print the coordinates in DS9 region file
# format so you can show them on the GX 339-4
# images
astro_coords = SkyCoord(np.array(astro_coords), unit=(un.deg, un.deg))
for a, astro in enumerate(astro_coords):
    text = ('circle({0:.5f},{1:.5f},5.000") '
            '# color=magenta '
            'text={{{2}}}').format(astro.ra.deg,
                                   astro.dec.deg, a)
    print(text)
    
    text = ('circle({0:.5f},{1:.5f},70.000") '
            '# color=cyan').format(astro.ra.deg,
                                   astro.dec.deg)
    print(text)

In [ ]:
num_iter = 5000
num_sources = np.shape(astro_coords)[0]
min_sep = 10./60./60.

# The pulsar and GX339 coordinates
# in case you need them
psr_coords = SkyCoord(np.array([['17h03m54.53', '-48d52m01.0']]))
gx_psr_coords = np.array([[psr_coords.ra.deg[0], psr_coords.dec.deg[0]],
                          [255.70575354295, -48.78976826464]])

In [ ]:
random_array_ras = np.ones((num_sources, num_iter))
random_array_decs = np.ones((num_sources, num_iter))

for r, row in enumerate(astro_coords):
    random_array_ras[r] = np.random.normal(loc=row.ra.deg, scale=0.4/60./60., size=num_iter)
    random_array_decs[r] = np.random.normal(loc=row.dec.deg, scale=0.4/60./60., size=num_iter)

In [ ]:
random_target_ras = np.ones((num_sources, num_iter))
random_target_decs = np.ones((num_sources, num_iter))

target_srlf = ('/raid/driessen/FlareStars/'
               'GX339/DeepImages/'
               'gx_deep_CORRCOL_DDE.app.restored_edited.pybdsm.srl')
target_srl = pd.read_csv(target_srlf, header=4)
target_srl.columns = target_srl.columns.str.replace(' ', '')
target_coordinates = SkyCoord(target_srl['RA']*un.deg,
                              target_srl['DEC']*un.deg)
target_raw_coordinates = np.vstack((target_srl['RA'], target_srl['DEC'])).T

for c in range(len(random_array_ras[0])):
    astro_coords_array = np.ones((num_sources, 2))
    
    astro_coords_array[:, 0] = random_array_ras[:, c]
    astro_coords_array[:, 1] = random_array_decs[:, c]
    
    astro_coords_array_sc = SkyCoord(np.array(astro_coords_array), unit=(un.deg, un.deg))
    
    A_tf, av_seps, target_new_coords = get_transformation(astro_coords_array_sc,
                                       target_coordinates,
                                       min_sep=min_sep)
    
    target_transformed = A_tf(target_raw_coordinates)
    
    random_target_ras[:, c] = target_new_coords[:, 0]
    random_target_decs[:, c] = target_new_coords[:, 1]
print('Done')

In [ ]:
for r, row in enumerate(random_target_ras):
    print('RA mean: {0:.6f} std: {1:3f} asec'.format(np.mean(row),
                                              np.std(row)*60.*60.))
    print('Dec mean: {0:.6f} std {1:3f} asec'.format(np.mean(random_target_decs[r]),
                                              np.std(random_target_decs[r])*60.*60.))
    print('--------------')

0.4 arcsec still dominates the uncertainties, so we are going to use 0.4 arcsec as our uncertainty in RA and DEC for our interesting sources.

Here we take the coordinates of every source in the deep stack GX 339-4 image, calculate the transformation, and apply the transformation.

In [ ]:
# Find and apply the transformations
# for just the deep GX 339-4 image

target_srlf = ('/raid/driessen/FlareStars/'
               'GX339/DeepImages/'
               'gx_deep_CORRCOL_DDE.app.restored_edited.pybdsm.srl')
target_srl = pd.read_csv(target_srlf, header=4)
target_srl.columns = target_srl.columns.str.replace(' ', '')
target_coordinates = SkyCoord(target_srl['RA']*un.deg,
                              target_srl['DEC']*un.deg)

average_separations = []
for s, field_srlf in enumerate([target_srlf]):
    bn = field_srlf.split('.srl')[0]
    bn = bn.split('/')[-1]
    
    new_srlf_name = ('/raid/driessen/FlareStars/'
                     'GX339/MFS_images/'
                     '{0}_tf.srl').format(bn)
    
    field_srl = pd.read_csv(field_srlf, header=4)
    field_srl.columns = field_srl.columns.str.replace(' ', '')
    try:
        field_coordinates = SkyCoord(field_srl['RA']*un.deg,
                                     field_srl['DEC']*un.deg)
        field_raw_coordinates = np.vstack((field_srl['RA'],
                                           field_srl['DEC'])).T

        (A_tf,
         av_seps,
         match_coords) = get_transformation(astro_coords,
                                            field_coordinates,
                                            min_sep=2./60./60.,
                                            print_outs=True)
        
        average_separations.append(av_seps)

        field_transformed = A_tf(field_raw_coordinates)

        field_srl['RA_tf'] = field_transformed[:, 0]
        field_srl['DEC_tf'] = field_transformed[:, 1]
        field_srl.to_csv(new_srlf_name, index=False)
    except KeyError:
        print('***********************************')
        print(field_srlf)
        print('***********************************')
    
    print('-------------------------------')
    print('-------------------------------')
average_separations = np.array(average_separations)

Next we get out the coordinates of our sources of interest, and double check that the 0.4 arcsec still dominates the uncertainties.

In [ ]:
# Read the updated coordinate file of the deep
# GX 339-4 image
deep_srl = pd.read_csv(('/raid/driessen/FlareStars/'
                        'GX339/MFS_images/'
                        'gx_deep_CORRCOL_DDE.app.'
                        'restored_edited.pybdsm_tf.srl'))
deep_srl_coords = SkyCoord(deep_srl['RA_tf']*un.deg, deep_srl['DEC_tf']*un.deg)

print(('Mean RA uncertainty for all sources: '
       '{:.4f}asec').format(np.mean(deep_srl['E_RA']*60.*60.)))
print(('Mean Dec uncertainty for all sources: '
       '{:.4f}asec').format(np.mean(deep_srl['E_DEC']*60.*60.)))

In [ ]:
# Read your old coordinates for the interesting
# sources
long_is = pd.read_csv(('/raid/driessen/FlareStars/'
                       'GX339/LightCurves_SN3/'
                       '2021.02.02_LongVariableSources.csv'))
long_is_coords = SkyCoord(long_is['ra_deg']*un.deg,
                          long_is['dec_deg']*un.deg)

In [ ]:
# Add the transformed, deep image coordinates
# to the information about each long-term
# variable source

new_coord_info = []

for l, lis in enumerate(long_is_coords):
    seps = lis.separation(deep_srl_coords)
    
    if np.nanmin(seps.deg) < 2./60./60.:
        print('{0:.5f} {1:.5f}'.format(lis.ra.deg,
                                       lis.dec.deg))
        print('{0:.5f} {1:.5f}'.format(deep_srl_coords[np.nanargmin(seps.deg)].ra.deg,
                                       deep_srl_coords[np.nanargmin(seps.deg)].dec.deg))
        print(('Separation between old '
               'and new coordinates: '
               '{:.6f} arcsec').format(np.nanmin(seps.deg)*60.*60.))
        new_coord_info.append([deep_srl.iloc[np.nanargmin(seps.deg)]['RA_tf'],
                               deep_srl.iloc[np.nanargmin(seps.deg)]['E_RA'],
                               deep_srl.iloc[np.nanargmin(seps.deg)]['DEC_tf'],
                               deep_srl.iloc[np.nanargmin(seps.deg)]['E_DEC']])
        print('-----------------------------')
new_coord_info = np.array(new_coord_info)

# Add the new information to the DataFrame
# and save it
long_is['RA_tf_deg'] = new_coord_info[:, 0]
long_is['E_RA_deg'] = new_coord_info[:, 1]
long_is['DEC_tf_deg'] = new_coord_info[:, 2]
long_is['E_DEC_deg'] = new_coord_info[:, 3]

long_is['E_RA_tf_deg'] = 0.4/60./60.
long_is['E_DEC_tf_deg'] = 0.4/60./60.

long_is.to_csv(('/raid/driessen/'
                'FlareStars/GX339/'
                'MFS_images/'
                '2021.02.11_'
                'LongVariableSources.csv'),
               index=False)

print('*****************')
print('Done')
print('*****************')

In [ ]:
print(('Mean RA uncertainty '
       'for long variable '
       'sources: {:.4f}asec').format(np.mean(long_is['E_RA_deg'])*60.*60))
print(('Mean DEC uncertainty '
       'for long variable '
       'sources: {:.4f}asec').format(np.mean(long_is['E_DEC_deg'])*60.*60))
print('----------------')
print(('Max RA uncertainty '
       'for long variable '
       'sources: {:.4f}asec').format(np.max(long_is['E_RA_deg'])*60.*60))
print(('Max DEC uncertainty '
       'for long variable '
       'sources: {:.4f}asec').format(np.max(long_is['E_DEC_deg'])*60.*60))
print('----------------')
print('----------------')
print(('Mean RA uncertainty '
       'for long variable '
       'sources after shifting: {:.4f}asec').format(np.mean(long_is['E_RA_tf_deg'])*60.*60))
print(('Mean DEC uncertainty '
       'for long variable '
       'sources after shifting: {:.4f}asec').format(np.mean(long_is['E_DEC_tf_deg'])*60.*60))

So we can see that 0.4 arcsec is the dominant uncertainty, so we will use this to perform our source matches.

Below is a demo of how we would calculate and apply the transformations to every epoch of the GX 339-4 ThunderKAT observations.

In [ ]:
# Find the transformations for each epoch of
# the GX 339-4 observations and apply these.

# All of the pyBDSF srl files
srlfs = glob.glob(('/raid/driessen/'
                   'FlareStars/GX339/'
                   'LightCurves_SN3/'
                   'SRLs/20*.srl'))

average_separations = []
for s, field_srlf in enumerate(srlfs):
    # Set up the name for the file
    # you'll save the position info
    # as
    bn = field_srlf.split('.srl')[0]
    bn = bn.split('/')[-1]
    new_srlf_name = ('/raid/driessen/FlareStars/'
                     'GX339/MFS_images/'
                     '{0}_tf.srl').format(bn)

    # Read in the srl file
    field_srl = pd.read_csv(field_srlf, header=4)
    field_srl.columns = field_srl.columns.str.replace(' ', '')
    try:
        field_coordinates = SkyCoord(field_srl['RA']*un.deg,
                                     field_srl['DEC']*un.deg)
        field_raw_coordinates = np.vstack((field_srl['RA'], field_srl['DEC'])).T

        # Get the transformation
        (A_tf,
         av_seps,
         match_coords) = get_transformation(astro_coords,
                                            field_coordinates,
                                            min_sep=2./60./60.,
                                            print_outs=False)
        
        average_separations.append(av_seps)

        # Transform all of the sources in the srl file
        field_transformed = A_tf(field_raw_coordinates)

        # Save the transformed source coordinates
        field_srl['RA_tf'] = field_transformed[:, 0]
        field_srl['DEC_tf'] = field_transformed[:, 1]
        field_srl.to_csv(new_srlf_name)
    except KeyError:
        print('***********************************')
        print(field_srlf)
        print('***********************************')
    
    print('-------------------------------')
    print('-------------------------------')
average_separations = np.array(average_separations)